# Проект на тему “Методы решения задачи о ранце со многими ограничениями”

## Полиномиальный приблизительный алгоритм

### GREEDYS

In [ ]:
def greedys(n, p, w, z, y, i, c):
  for j in range(n):
    if y[j] == -1 and w[j] <= c:
      y[j] = i
      c -= w[j]
      z += p[j]
  return z, y

### Опциональная функция для контроля работы алгоритма

In [ ]:
def test_current_step(step, z, y):
  print('Step:', step)
  print('z =', z)
  print('y =', y)
  print()

### MTHM

In [ ]:
import timeit

def MTHM(n, m, p, w, c):

  starttime = timeit.default_timer()

  # initial solution
  z = 0
  y = [-1 for j in range(n)]
  for i in range(m):
    z, y = greedys(n, p, w, z, y, i, c[i]) # нет смысла заводить C штрих, так как каждый раз подается c[i], который изменяется в greedys, но не возвращается

  # test_current_step(1, z, y)

  # rearrangement
  z = 0
  c_modified = c.copy()
  i = 0
  for j in range(n - 1, -1, -1):
    if y[j] > -1:
      l = -1
      for k in range(i, m): # странный момент псевдокода
        if w[j] <= c_modified[k]:
          l = k
          break
      if l == -1:
        for k in range(0, i):
          if w[j] <= c_modified[k]:
            l = k
            break
      if l == -1:
        y[j] = -1
      else:
        y[j] = l
        c_modified[l] -= w[j]
        z += p[j]
        if l < m:
          i = l + 1
        else:
          i = 0
  for i in range(m):
    z, y = greedys(n, p, w, z, y, i, c_modified[i])
  
  # test_current_step(2, z, y)
  
  # first improvement
  for j in range(n):
    if y[j] > -1:
      for k in range(j + 1, n):
        if -1 < y[k] and y[k] != y[j]:
          if w[j] > w[k]:
            h = j
            l = k
          else:
            h = k
            l = j
          d = w[h] - w[l]
          min_w = -1
          for ii in range(n):
            if y[ii] == -1:
              if min_w == -1 or min_w > w[ii]:
                min_w = w[ii]
          if d <= c_modified[y[l]] and c_modified[y[h]] + d >= min_w:
            max_p = -1
            for ii in range(n):
              if y[ii] == -1 and w[ii] <= c_modified[y[h]] + d:
                if max_p == -1 or max_p < p[ii]:
                  max_p = p[ii]
                  t = ii
            c_modified[y[h]] = c_modified[y[h]] + d - w[t]
            c_modified[y[l]] -= d
            y[t] = y[h]
            y[h] = y[l]
            y[l] = y[t]
            z += p[t]
  
  # test_current_step(3, z, y)
  
  # second improvement
  for j in range(n - 1, -1, -1):
    if y[j] > -1:
      c_temp = c_modified[y[j]] + w[j]
      Y = list()
      for k in range(n):
        if y[k] == -1 and w[k] <= c_temp:
          Y.append(k)
          c_temp -= w[k]
      sum_of_Y_elems = 0
      for el in Y:
        sum_of_Y_elems += p[el]
      if sum_of_Y_elems > p[j]:
        for el in Y:
          y[el] = y[j]
        c_modified[y[j]] = c_temp
        y[j] = -1
        z += sum_of_Y_elems - p[j]

  return z, y, timeit.default_timer() - starttime
  

### Проверка работы алгоритма

In [ ]:
def test_MTHM_with_print(n, m, p, w, c):
  max_price, best_permutation, necessary_time = MTHM(n, m, p, w, c)

  print('Best permutation:')
  best_permutation = list(best_permutation)
  for i in range(m):
    knapsack = []
    for j in range(n):
      if best_permutation[j] == i:
        knapsack.append(1)
      else:
        knapsack.append(0)
    print(i + 1, 'knapsack:', knapsack)

  print('Price of best permutation:', max_price)

  # При больших m и n слишком долго запускать алгоритм 10 раз, тем не менее удалось выяснить, что разброс по времени выполнения у алгоритмов небольшой

  total_time = necessary_time
  number_of_runs = 10
  for i in range(number_of_runs - 1):
    total_time += MTHM(n, m, p, w, c)[2]
  necessary_time = total_time / number_of_runs
  print('Average time:', necessary_time)

  return max_price, necessary_time

In [ ]:
n = 10 # число предметов
m = 2 # число рюкзаков
p = [78, 35, 89, 36, 94, 75, 74, 79, 80, 16] # ценности предметов
w = [18, 9, 23, 20, 59, 61, 70, 75, 76, 30] # массы предметов
c = [103, 156] # размеры рюкзаков

algo_results = {}

algo_results['first_algo_res_first_test'], algo_results['first_algo_time_first_test'] = test_MTHM_with_print(n, m, p, w, c)

# Вывод искомого алгоритма без обработки: (423, [1, 0, 1, 1, 0, 1, -1, -1, -1, 0])

Best permutation:
1 knapsack: [0, 1, 0, 0, 1, 0, 0, 0, 0, 1]
2 knapsack: [1, 0, 1, 1, 0, 1, 0, 0, 0, 0]
Price of best permutation: 423
Average time: 8.772079997925175e-05


In [ ]:
n = 9
m = 2
p = [80, 20, 60, 40, 60, 60, 65, 25, 30]
w = [40, 10, 40, 30, 50, 50, 55, 25, 40]
c = [100, 150]

algo_results['first_algo_res_second_test'], algo_results['first_algo_time_second_test'] = test_MTHM_with_print(n, m, p, w, c)

# Вывод искомого алгоритма без обработки: (350, [0, 0, 1, 1, -1, 0, 1, 1, -1])

Best permutation:
1 knapsack: [1, 1, 0, 0, 0, 1, 0, 0, 0]
2 knapsack: [0, 0, 1, 1, 0, 0, 1, 1, 0]
Price of best permutation: 350
Average time: 9.033890003138368e-05


In [ ]:
n = 6
m = 2
p = [110, 150, 70, 80, 30, 5]
w = [40, 60, 30, 40, 20, 5]
c = [65, 85]

algo_results['first_algo_res_third_test'], algo_results['first_algo_time_third_test'] = test_MTHM_with_print(n, m, p, w, c)

Best permutation:
1 knapsack: [0, 1, 0, 0, 0, 1]
2 knapsack: [1, 0, 0, 1, 0, 0]
Price of best permutation: 345
Average time: 5.234689999724651e-05


## Bound and bound алгоритм

### UPPER

In [ ]:
def upper(n, m, p, w, c, x_caret, S, i):
  sum_1 = 0
  sum_2 = 0
  for j in S[i]:
    sum_1 += w[j] * x_caret[i][j]
  for k in range(i + 1, m):
    sum_2 += c[k]
  c_ = c[i] - sum_1 + sum_2

  N = list()
  for j in range(n):
    all_x_equal_to_0 = True
    for k in range(0, i + 1):
      if x_caret[k][j] != 0:
        all_x_equal_to_0 = False
        break
    if all_x_equal_to_0 == True:
      N.append(j)

  w_ = []
  p_ = []
  for el in N:
    w_.append(w[el])
    p_.append(p[el])
  z = dynamic_method(N, len(N), w_, p_, c_, False)

  sum_3 = 0
  for k in range(0, i + 1):
    for j in S[k]:
      sum_3 += p[j] * x_caret[k][j]
  U = sum_3 + z

  return U

### LOWER

In [ ]:
def lower(n, m, p, w, c, x_caret, S, i):
  L = 0
  for k in range(0, i + 1):
    for j in S[k]:
      L += p[j] * x_caret[k][j]
  
  N_ = list()
  for j in range(n):
    all_x_equal_to_0 = True
    for k in range(0, i + 1):
      if x_caret[k][j] != 0:
        all_x_equal_to_0 = False
        break
    if all_x_equal_to_0 == True:
      N_.append(j)

  N = set(N_) - set(S[i])

  sum_1 = 0
  for j in S[i]:
    sum_1 += w[j] * x_caret[i][j]
  c_ = c[i] - sum_1
  k = i
  x_tilda = [[0 for j in range(n)] for r in range(m)] # m x n
  for r in range(m):
        for j in range(n):
          x_tilda[r][j] = x_caret[r][j]

  while k <= m - 1:
    w_ = []
    p_ = []
    for el in N:
      w_.append(w[el])
      p_.append(p[el])
    z, used_indexes = dynamic_method(list(N), len(N), w_, p_, c_, True)
    for el in used_indexes:
      x_tilda[k][el] = 1

    L += z
    N_ = set(N_) - set(used_indexes)
    N = N_
    k += 1
    if k != m:
      c_ = c[k]

  return L, x_tilda

### MTM

In [ ]:
import timeit

def MTM(n, m, p, w, c):

  starttime = timeit.default_timer()

  # initialize
  S = [[] for k in range(m)]
  x_caret = [[0 for j in range(n)] for k in range(m)]
  x = [[0 for j in range(n)] for k in range(m)]
  z = 0
  i = 0
  U = upper(n, m, p, w, c, x_caret, S, i)
  UB = U
  return_to_2_step = True

  # heuristic
  while return_to_2_step:
    L, x_tilda = lower(n, m, p, w, c, x_caret, S, i)
    go_to_backtrack = False
    if L > z:
      z = L
      for k in range(m):
        for j in range(n):
          x[k][j] = x_caret[k][j]
      for k in range(i, m):
        for j in range(n):
          if x_tilda[k][j] == 1:
            x[k][j] = 1
      if z == UB:
        return z, x, timeit.default_timer() - starttime
      if z == U:
        go_to_backtrack = True

    # define a new current solution
    if not go_to_backtrack:
      worse_solution_flag = False
      while i != m - 1 and not worse_solution_flag:
        I = set()
        for l in range(n):
          if x_tilda[i][l] == 1:
            I.add(l)
        while len(I) > 0:
          j = min(I)
          I -= {j}
          if j not in S[i]:
            S[i].append(j)
          x_caret[i][j] = 1
          U = upper(n, m, p, w, c, x_caret, S, i)
          if U <= z:
            worse_solution_flag = True
            break
        i += 1
      if not worse_solution_flag:
        i = m - 2
      else:
        i -= 1

    return_to_2_step = False
    while i > -1:
      while len(S[i]) > 0:
        j = S[i][len(S[i]) - 1]
        if x_caret[i][j] == 0:
          S[i].pop(len(S[i]) - 1)
        else:
          x_caret[i][j] = 0
          U = upper(n, m, p, w, c, x_caret, S, i)
          if U > z:
            return_to_2_step = True
            break
      if return_to_2_step:
        break
      i -= 1

  return z, x, timeit.default_timer() - starttime


### Классическая задача о ранце

In [ ]:
matrix = [] # глобальная переменная, иначе рекурсивная функция будет неоптимальна для использования
ans = []

In [ ]:
def dynamic_method(ids, n, weights, p, c, restore_ans, type_of_func = 1):
  global matrix, ans
  if type_of_func == 1:
    restore_answer = restore_answer_with_priority_to_skip_item_if_possible
  else:
    restore_answer = restore_answer_with_priority_to_take_item_if_possible
  matrix = [[0 for j in range(c + 1)] for i in range(n + 1)] # i - сколько предметов задействовали, j - вес рюкзака
  ans = []

  for i in range(1, n + 1):
    for j in range(c + 1):
      if weights[i - 1] > j: # сравниваем вес i-го предмета и вместимость рюкзака (чтобы понять можем ли мы вместить туда этот предмет)
        matrix[i][j] = matrix[i - 1][j] # если не может, то лучший результат такой же, как и у (i-1) предметов
      else:
        matrix[i][j] = max(matrix[i - 1][j], matrix[i - 1][j - weights[i - 1]] + p[i - 1]) # если может, то сравниваем показатели если брать и не брать этот предмет

  if restore_ans:
    restore_answer(n, c, weights)
    items = []
    for id in ans:
      items.append(ids[id - 1])
    return matrix[n][c], items
  else:
    return matrix[n][c]

In [ ]:
# в силу того, что порой один и тот же вес можно набрать двумя разными способами, возникают две вариации функции восстановления ответа

# вариация восстановления ответа с приоритетом пропуска элемента при возможности не брать его
def restore_answer_with_priority_to_skip_item_if_possible(i, j, weights):
  global matrix, ans
  if matrix[i][j] == 0:
    return
  if matrix[i - 1][j] == matrix[i][j]:
    restore_answer_with_priority_to_skip_item_if_possible(i - 1, j, weights)
  else:
    restore_answer_with_priority_to_skip_item_if_possible(i - 1, j - weights[i - 1], weights)
    ans.append(i)

# вариация восстановления ответа с приоритетом добавления элемента в ответ при возможности его вместить
def restore_answer_with_priority_to_take_item_if_possible(i, j, weights):
  global matrix, ans
  if matrix[i][j] == 0:
    return
  if matrix[i][j] == matrix[i - 1][j - weights[i - 1]] + p[i - 1]:
    restore_answer_with_priority_to_take_item_if_possible(i - 1, j - weights[i - 1], weights)
    ans.append(i)
  else:
    restore_answer_with_priority_to_take_item_if_possible(i - 1, j, weights)

### Тестирование динамического решения классического рюкзака

In [ ]:
n = 4
w = [5, 10, 6, 5]
p = [3, 5, 4, 2]
c = 14
ids = [4, 5, 9, 177]
print('Priority to skip item:', dynamic_method(ids, n, w, p, c, True, 1))
print('Priority to take item:', dynamic_method(ids, n, w, p, c, True, 0))

# Правильный ответ: 7; 1 и 3, то есть 4 и 9.

Priority to skip item: (7, [4, 9])
Priority to take item: (7, [4, 9])


In [ ]:
n = 5
w = [3, 4, 5, 8, 9]
p = [1, 6, 4, 7, 6]
c = 13
ids = [0, 1, 2, 3]
print('Priority to skip item:', dynamic_method(ids, n, w, p, c, True, 1))
print('Priority to take item:', dynamic_method(ids, n, w, p, c, True, 0))

# Правильный ответ: 13; 2 и 4, то есть 1 и 3.

Priority to skip item: (13, [1, 3])
Priority to take item: (13, [1, 3])


In [ ]:
n = 10
c = 165
w = [23, 31, 29, 44, 53, 38, 63, 85, 89, 82]
p = [92, 57, 49, 68, 60, 43, 67, 84, 87, 72]
ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print('Priority to skip item:', dynamic_method(ids, n, w, p, c, True, 1))
print('Priority to take item:', dynamic_method(ids, n, w, p, c, True, 0))

# Правильный ответ: 309; 1, 2, 3, 4 и 6, то есть 0, 1, 2, 3 и 5.

Priority to skip item: (309, [0, 1, 2, 3, 5])
Priority to take item: (309, [0, 1, 2, 3, 5])


### Тестирование одной из возникающих неоднозначных ситуаций при выполнении теста №1. Подходят сразу 2 варианта укладки рюкзака, выбранный нами ответ зависит от выбора вариации восстановления ответа.

In [ ]:
n = 7
p = (35, 36, 75, 74, 79, 80, 16) # 35+75+80 = 190 = 36+75+79
w = (9, 20, 61, 70, 75, 76, 30) # 9+61+76= 146 < 156; 20+61+75=156;
c = (156)
ids = [1, 3, 5, 6, 7, 8, 9] 

print('Priority to skip item:', dynamic_method(ids, n, w, p, c, True, 1))
print('Priority to take item:', dynamic_method(ids, n, w, p, c, True, 0))

# решение взять 1й 3й и 6й = решение взять 2й 3й и 5й
# то есть [1,5,8] = [3,5,7]

Priority to skip item: (190, [3, 5, 7])
Priority to take item: (190, [1, 5, 8])


### Проверка работы алгоритма

In [ ]:
def test_MTM_with_print(n, m, p, w, c):
  max_price, best_permutation, necessary_time = MTM(n, m, p, w, c)

  print('Best permutation:')
  best_permutation = list(best_permutation)
  for i in range(m):
    print(i + 1, 'knapsack:', best_permutation[i])

  print('Price of best permutation:', max_price)

  # При больших m и n слишком долго запускать алгоритм 10 раз, тем не менее удалось выяснить, что разброс по времени выполнения у алгоритмов небольшой

  total_time = necessary_time
  number_of_runs = 10
  for i in range(number_of_runs - 1):
    total_time += MTM(n, m, p, w, c)[2]
  necessary_time = total_time / number_of_runs
  print('Average time:', necessary_time)

  return max_price, necessary_time

In [ ]:
n = 10
m = 2
p = [78, 35, 89, 36, 94, 75, 74, 79, 80, 16]
w = [18, 9, 23, 20, 59, 61, 70, 75, 76, 30]
c = [103, 156]

algo_results['second_algo_res_first_test'], algo_results['second_algo_time_first_test'] = test_MTM_with_print(n, m, p, w, c)

# Вывод искомого алгоритма без обработки: (452, [[1, 0, 1, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0, 1, 0]])

Best permutation:
1 knapsack: [1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
2 knapsack: [0, 0, 0, 1, 1, 0, 0, 0, 1, 0]
Price of best permutation: 452
Average time: 0.011779578299979221


In [ ]:
n = 9
m = 2
p = [80, 20, 60, 40, 60, 60, 65, 25, 30]
w = [40, 10, 40, 30, 50, 50, 55, 25, 40]
c = [100, 150]

algo_results['second_algo_res_second_test'], algo_results['second_algo_time_second_test'] = test_MTM_with_print(n, m, p, w, c)

# Вывод искомого алгоритма без обработки: (350, [[1, 1, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 1, 1, 0]])

Best permutation:
1 knapsack: [1, 1, 0, 0, 1, 0, 0, 0, 0]
2 knapsack: [0, 0, 1, 1, 0, 0, 1, 1, 0]
Price of best permutation: 350
Average time: 0.006739554499995392


In [ ]:
n = 6
m = 2
p = [110, 150, 70, 80, 30, 5]
w = [40, 60, 30, 40, 20, 5]
c = [65, 85]

algo_results['second_algo_res_third_test'], algo_results['second_algo_time_third_test'] = test_MTM_with_print(n, m, p, w, c)

Best permutation:
1 knapsack: [0, 1, 0, 0, 0, 1]
2 knapsack: [1, 0, 0, 1, 0, 0]
Price of best permutation: 345
Average time: 0.01753894280006989


## Перебор всех возможных вариантов укладки

### Время, необходмое для генерации перестановки длины X

In [ ]:
from itertools import product
import time

def time_to_generate_all_permutations_with_repetitions(size):
  start = time.time()
  #print('Permutations with repetitions:')
  #print(list(product([0, 1], repeat=size)))
  for el in product([0, 1], repeat=size):
    pass
  end = time.time()
  print('Time:', end - start)

In [ ]:
time_to_generate_all_permutations_with_repetitions(10)

Time: 0.00010228157043457031


In [ ]:
time_to_generate_all_permutations_with_repetitions(20)

Time: 0.1273057460784912


### Функция для перебора всех возможных вариантов

In [ ]:
from itertools import product

def fits(c, w, perm):
  sum_of_w = 0
  for i in range(len(perm)):
    sum_of_w += w[i] * perm[i]
    if sum_of_w > c:
      return False
  return True


def check_all_permutations_with_repetitions(n, m, p, w, c):
  starttime = timeit.default_timer()
  size = n * m
  elements = [0, 1]
  max_sum = -1
  best_perm = []

  for perm in product(elements, repeat=size): # перебираем все возможные комбинации 0 и 1
    # проверим хватит ли места в рюкзаках
    are_fitting = True
    for i in range(m):
      if fits(c[i], w, perm[i*n : (i+1)*n]) is False:
        are_fitting = False
        break
    if are_fitting is False: # если предметы не влезают в один из рюкзаков переходим к следующей перестановке
      continue
  
    # проверим, что премдет не лежит в разных рюкзаках
    are_items_in_diff_knapsacks = False
    are_items_in_knapsacks = [] # понадобится для подсчета стоимости предметов в рюкзаках, предполагается, что состоит из 0 и 1
    # если в are_items_in_knapsacks попадут значения больше 1, то мы не попадем в !are_in_diff_knapsacks
    for i in range(n): # количество предметов в рюкзаке
      how_many_knapsacks_the_item_is_in = 0 # в скольки рюкзаках лежит предмет
      for j in range(m): # количество рюкзаков
        how_many_knapsacks_the_item_is_in += perm[j*n + i]
      if how_many_knapsacks_the_item_is_in > 1:
        are_items_in_diff_knapsacks = True
        break
      are_items_in_knapsacks.append(how_many_knapsacks_the_item_is_in)

    # предметы лежат в разных рюкзаках и влезают в них
    if are_items_in_diff_knapsacks is False:
      sum_price = 0
      for i in range(n): # количество предметов в рюкзаке
        sum_price += are_items_in_knapsacks[i] * p[i]
      if sum_price > max_sum:
        max_sum = sum_price
        best_perm = perm
  
  return best_perm, max_sum, timeit.default_timer() - starttime
        

### Проверка функции на тестах предыдущих алгоритмов

In [ ]:
def test_all_perm_algo_with_print(n, m, p, w, c):
  best_permutation, max_price, necessary_time = check_all_permutations_with_repetitions(n, m, p, w, c)

  print('Best permutation:')
  best_permutation = list(best_permutation)
  for i in range(m):
    print(i + 1, 'knapsack:', best_permutation[i*n : (i+1)*n])
  
  print('Price of best permutation:', max_price)

  # При больших m и n слишком долго запускать алгоритм 10 раз, тем не менее удалось выяснить, что разброс по времени выполнения у алгоритмов небольшой

  total_time = necessary_time
  number_of_runs = 10
  for i in range(number_of_runs - 1):
    total_time += check_all_permutations_with_repetitions(n, m, p, w, c)[2]
  necessary_time = total_time / number_of_runs
  print('Average time:', necessary_time)

  return max_price, necessary_time

In [ ]:
n = 10
m = 2
p = [78, 35, 89, 36, 94, 75, 74, 79, 80, 16]
w = [18, 9, 23, 20, 59, 61, 70, 75, 76, 30]
c = [103, 156]

algo_results['third_algo_res_first_test'], algo_results['third_algo_time_first_test'] = test_all_perm_algo_with_print(n, m, p, w, c)

Best permutation:
1 knapsack: [0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
2 knapsack: [1, 0, 0, 0, 0, 1, 0, 0, 1, 0]
Price of best permutation: 452
Average time: 2.092283210999949


In [ ]:
n = 9
m = 2
p = [80, 20, 60, 40, 60, 60, 65, 25, 30]
w = [40, 10, 40, 30, 50, 50, 55, 25, 40]
c = [100, 150]

algo_results['third_algo_res_second_test'], algo_results['third_algo_time_second_test'] = test_all_perm_algo_with_print(n, m, p, w, c)

Best permutation:
1 knapsack: [0, 1, 1, 0, 0, 1, 0, 0, 0]
2 knapsack: [1, 0, 0, 1, 0, 0, 1, 1, 0]
Price of best permutation: 350
Average time: 0.5682243501000357


In [ ]:
n = 6
m = 2
p = [110, 150, 70, 80, 30, 5]
w = [40, 60, 30, 40, 20, 5]
c = [65, 85]

algo_results['third_algo_res_third_test'], algo_results['third_algo_time_third_test'] = test_all_perm_algo_with_print(n, m, p, w, c)

Best permutation:
1 knapsack: [0, 1, 0, 0, 0, 0]
2 knapsack: [1, 0, 0, 1, 0, 1]
Price of best permutation: 345
Average time: 0.009350274499956868


## Сравниваем время работы и результаты алгоритмов для тестов из учебника

In [ ]:
# Очевидно, что мы работаем с малыми m и n

import pandas as pd

index = ['MTHM', 'MTM', 'All permutations algorithm']
columns = ['Result_1', 'Time_1', 'Result_2', 'Time_2', 'Result_3', 'Time_3']
algo_number = ['first', 'second', 'third']
test_number = ['first', 'second', 'third']
num_of_tests = 3
num_of_algo = 3

df = pd.DataFrame(0, index=index, columns=columns)

for i in range(num_of_algo):
  for j in range(1, num_of_tests + 1):
    df.loc[index[i], 'Result_' + str(j)] = algo_results[algo_number[i] + '_algo_res_' + test_number[j - 1] + '_test']
    df.loc[index[i], 'Time_' + str(j)] = algo_results[algo_number[i] + '_algo_time_' + test_number[j - 1] + '_test']

# pd.set_option("display.max_rows", None, "display.max_columns", None)
print(df.to_string())
print()

index2 = ['Test_1', 'Test_2', 'Test_3']
columns2 = ['n', 'm']
n_s = [10, 9, 6]
m_s = [2, 2, 2]
df2 = pd.DataFrame(0, index=index2, columns=columns2)

for i in range(num_of_tests):
  df2.loc['Test_' + str(i + 1), 'n'] = n_s[i]
  df2.loc['Test_' + str(i + 1), 'm'] = m_s[i]

print(df2.to_string())

                            Result_1    Time_1  Result_2    Time_2  Result_3    Time_3
MTHM                             423  0.000088       350  0.000090       345  0.000052
MTM                              452  0.011780       350  0.006740       345  0.017539
All permutations algorithm       452  2.092283       350  0.568224       345  0.009350

         n  m
Test_1  10  2
Test_2   9  2
Test_3   6  2


## Генерация тестов

### Важно помнить, что алгоритмы работают на отсортированных по ценностям предметах и по размернсти рюкзаках.

In [ ]:
def sorting_by_importance(p, w):
  importance = []
  for i in range(len(p)):
    importance.append((i, p[i]/w[i]))
  importance.sort(reverse=True, key = lambda x: x[1])
  permutation = [elem[0] for elem in importance]
  return [p[i] for i in permutation], [w[i] for i in permutation]

### Совершенно разные вместимости рюкзаков

In [ ]:
import random

n_list = [10, 15, 20, 25]
m_list = [2, 3, 4]

tests_dissimilar_capacities = list()

for k in range(len(n_list) * len(m_list)):
  p = []
  w = []
  c = []

  for i in range(n_list[k % 4]):
    p.append(random.randint(1, 1000))
    w.append(random.randint(1, 1000))
  p, w = sorting_by_importance(p, w)

  sum_of_w = sum(w)
  for i in range(m_list[k // 4] - 1):
    c.append(random.randint(0, round(0.5 * sum_of_w - sum(c))))
  c.append(round(0.5 * sum_of_w - sum(c)))
  c.sort()

  tests_dissimilar_capacities.append([n_list[k % 4], m_list[k // 4], p, w, c])

In [ ]:
print(tests_dissimilar_capacities[0])

[10, 2, [961, 982, 266, 671, 951, 263, 359, 244, 287, 212], [38, 48, 66, 241, 631, 177, 483, 462, 879, 757], [395, 1496]]


### Похожие вместимости рюкзаков

In [ ]:
tests_similar_capacities = list()

for k in range(len(n_list) * len(m_list)):
  p = []
  w = []
  c = []

  for i in range(n_list[k % 4]):
    p.append(random.randint(1, 1000))
    w.append(random.randint(1, 1000))
  p, w = sorting_by_importance(p, w)
  
  sum_of_w = sum(w)
  for i in range(m_list[k // 4] - 1):
    c.append(random.randint(round(0.4 * sum_of_w / m_list[k // 4]), round(0.6 * sum_of_w / m_list[k // 4])))
  c.append(round(0.5 * sum_of_w - sum(c)))
  c.sort()

  tests_similar_capacities.append([n_list[k % 4], m_list[k // 4], p, w, c])

In [ ]:
print(tests_similar_capacities[4])

[10, 3, [523, 828, 735, 758, 617, 661, 708, 121, 582, 464], [51, 183, 229, 699, 606, 810, 950, 164, 947, 961], [898, 907, 995]]


## Работоспособность алгоритмов на сгенерированных тестах

### Работа алгоритмов.

In [ ]:
from datetime import datetime

index = [i for i in range(len(n_list) * len(m_list))]
columns = ['m', 'n', 'MTHM_sim', 'MTM_sim', 'MTHM_dissim', 'MTM_dissim']

df2 = pd.DataFrame(0, index=index, columns=columns)

print("Current time:", datetime.now().strftime("%H:%M:%S"))
for i in range(len(n_list) * len(m_list)):
  input_sim_vals = tests_similar_capacities[i]
  input_dissim_vals = tests_dissimilar_capacities[i]
  df2.loc[i, 'm'] = tests_similar_capacities[i][1]
  df2.loc[i, 'n'] = tests_similar_capacities[i][0]
  price, _, time = MTHM(input_sim_vals[0], input_sim_vals[1], input_sim_vals[2], input_sim_vals[3], input_sim_vals[4])
  df2.loc[i, columns[2]] = str(time) + '(' + str(price) + ')'
  price, _, time = MTM(input_sim_vals[0], input_sim_vals[1], input_sim_vals[2], input_sim_vals[3], input_sim_vals[4])
  df2.loc[i, columns[3]] = str(time) + '(' + str(price) + ')'
  price, _, time = MTHM(input_dissim_vals[0], input_dissim_vals[1], input_dissim_vals[2], input_dissim_vals[3], input_dissim_vals[4])
  df2.loc[i, columns[4]] = str(time) + '(' + str(price) + ')'
  price, _, time = MTM(input_dissim_vals[0], input_dissim_vals[1], input_dissim_vals[2], input_dissim_vals[3], input_dissim_vals[4])
  df2.loc[i, columns[5]] = str(time) + '(' + str(price) + ')'
  print("Iteration done:", i, "Current time:", datetime.now().strftime("%H:%M:%S"))

print(df2.to_string())

Current time: 19:48:29
Iteration done: 0 Current time: 19:48:29
Iteration done: 1 Current time: 19:49:31
Iteration done: 2 Current time: 19:50:15
Iteration done: 3 Current time: 19:50:18
Iteration done: 4 Current time: 19:50:18
Iteration done: 5 Current time: 19:50:19
Iteration done: 6 Current time: 19:50:39
Iteration done: 7 Current time: 19:51:04
Iteration done: 8 Current time: 19:51:13
Iteration done: 9 Current time: 19:51:37
Iteration done: 10 Current time: 19:56:34
Iteration done: 11 Current time: 20:03:55
    m   n                       MTHM_sim                    MTM_sim                    MTHM_dissim                 MTM_dissim
0   2  10    6.334299996524351e-05(2842)  0.02178191000029983(2842)    6.170400001792586e-05(4453)  0.01781504100017628(4453)
1   2  15      9.3837999884272e-05(6192)    62.10917076700025(6192)   0.00011881599994012504(5251)  0.07638941999994131(5101)
2   2  20    0.0002645529998517304(8457)   43.426270691000354(8652)   0.00018695100015975186(7970)  0.129

### Работа алгоритмов без сортировки предметов и рюкзаков.



In [ ]:
from datetime import datetime

index = [i for i in range(len(n_list) * len(m_list))]
columns = ['m', 'n', 'MTHM_sim', 'MTM_sim', 'MTHM_dissim', 'MTM_dissim']

df2 = pd.DataFrame(0, index=index, columns=columns)

print("Current time:", datetime.now().strftime("%H:%M:%S"))
for i in range(len(n_list) * len(m_list)): # 12
  input_sim_vals = tests_similar_capacities[i]
  input_dissim_vals = tests_dissimilar_capacities[i]
  price, _, time = MTHM(input_sim_vals[0], input_sim_vals[1], input_sim_vals[2], input_sim_vals[3], input_sim_vals[4])
  df2.loc[i, columns[2]] = str(time) + '(' + str(price) + ')'
  price, _, time = MTM(input_sim_vals[0], input_sim_vals[1], input_sim_vals[2], input_sim_vals[3], input_sim_vals[4])
  df2.loc[i, columns[3]] = str(time) + '(' + str(price) + ')'
  price, _, time = MTHM(input_dissim_vals[0], input_dissim_vals[1], input_dissim_vals[2], input_dissim_vals[3], input_dissim_vals[4])
  df2.loc[i, columns[4]] = str(time) + '(' + str(price) + ')'
  price, _, time = MTM(input_dissim_vals[0], input_dissim_vals[1], input_dissim_vals[2], input_dissim_vals[3], input_dissim_vals[4])
  df2.loc[i, columns[5]] = str(time) + '(' + str(price) + ')'
  print("Iteration done:", i, "Current time:", datetime.now().strftime("%H:%M:%S"))

print(df2.to_string())

Current time: 21:09:30
Iteration done: 0 Current time: 21:09:31
Iteration done: 1 Current time: 21:09:32
Iteration done: 2 Current time: 21:10:32
Iteration done: 3 Current time: 21:11:03
Iteration done: 4 Current time: 21:11:03
Iteration done: 5 Current time: 21:11:29
Iteration done: 6 Current time: 21:12:48
Iteration done: 7 Current time: 21:12:56
Iteration done: 8 Current time: 21:13:01
Iteration done: 9 Current time: 21:15:43
Iteration done: 10 Current time: 22:22:32
Iteration done: 11 Current time: 23:31:38
    m  n                      MTHM_sim                     MTM_sim                   MTHM_dissim                  MTM_dissim
0   0  0   5.726200106437318e-05(2577)    1.0254777779991855(3168)   7.330100197577849e-05(3463)  0.024979212001198903(3765)
1   0  0   9.335099821328185e-05(4886)    0.7772602689983614(5631)   9.373500142828561e-05(5497)   0.06686601100227563(6361)
2   0  0  0.00022295999951893464(6061)      59.5125773419968(6366)   0.0001940709989867173(8152)    0.111286

In [ ]:
for i in range(len(n_list) * len(m_list)):
  df2.loc[i, 'm'] = tests_similar_capacities[i][1]
  df2.loc[i, 'n'] = tests_similar_capacities[i][0]

print(df2.to_string())

    m   n                      MTHM_sim                     MTM_sim                   MTHM_dissim                  MTM_dissim
0   2  10   5.726200106437318e-05(2577)    1.0254777779991855(3168)   7.330100197577849e-05(3463)  0.024979212001198903(3765)
1   2  15   9.335099821328185e-05(4886)    0.7772602689983614(5631)   9.373500142828561e-05(5497)   0.06686601100227563(6361)
2   2  20  0.00022295999951893464(6061)      59.5125773419968(6366)   0.0001940709989867173(8152)    0.1112868130003335(8734)
3   2  25   0.0009236429978045635(7231)    30.82943041999897(10009)  0.00018103099864674732(9652)  0.18311185600032331(12570)
4   3  10   8.598800195613876e-05(3566)  0.025371366999024758(3566)   6.264500188990496e-05(3373)  0.026804438002727693(3403)
5   3  15  0.00012464099927456118(5347)    14.128925872999389(6009)  0.00011496200022520497(2725)    12.523208187001728(3470)
6   3  20   0.0002062109997496009(7414)     0.843919034999999(7931)  0.00020299600146245211(6174)     77.3858462879979